## libraries and function 

In [ ]:
!pip install impyute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
import pandas as pd
import time 
!pip install missingpy
from missingpy import MissForest
import math

     |████████████████████████████████| 49 kB 3.5 MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### compute_err function 

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [ ]:
def compute_err_nuclear(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_nuclear = NuclearNormMinimization(max_iters=100).fit_transform(Xtr_nan)
    mus_nuclear = np.asarray([np.mean(Xtr_nuclear[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_nuclear = np.asarray([(sum(ytrain==g))*np.cov(Xtr_nuclear[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_nuclear = np.sum(S_nuclear, axis = 0)/len(ytrain)
    nuclear_err = err(mus, S, mus_nuclear, S_nuclear)
    nuclear_time = time.time()-start 

    e_rate.append(nuclear_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)    

In [ ]:
def compute_err_miss(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_mforest = MissForest(random_state=0).fit_transform(Xtr_nan)
    mus_mforest = np.asarray([np.mean(Xtr_mforest[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_mforest = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mforest[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mforest = np.sum(S_mforest, axis = 0)/len(ytrain)
    mforest_err = err(mus, S, mus_mforest, S_mforest)
    mforest_time = time.time()-start    

    e_rate.append(mforest_err)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate)    

# Heart

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header = None,sep=',')
print(data.head())
test = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.test',
                     header=None, sep = ',')
data = pd.concat([data, test])
data = data.to_numpy()
X,y = data[:,1:], data[:,0]
X = X.astype(np.float32)
G = len(np.unique(y)) 
print(np.shape(X))
for g in range(G):
  print(sum(y==g))

   0   1   2   3   4   5   6   7   8   ...  36  37  38  39  40  41  42  43  44
0   1  59  52  70  67  73  66  72  61  ...  56  62  56  72  62  74  74  64  67
1   1  72  62  69  67  78  82  74  65  ...  71  63  60  69  73  67  71  56  58
2   1  71  62  70  64  67  64  79  65  ...  70  66  65  64  55  61  41  51  46
3   1  69  71  70  78  61  63  67  65  ...  61  66  65  72  73  68  68  59  63
4   1  70  66  61  66  61  58  69  69  ...  69  70  66  70  64  60  55  49  41

[5 rows x 45 columns]
(267, 44)
55
212


In [ ]:
G = 2
miss_err = np.array([compute_err_miss(X, y, G, .2, runs = 10),
                    compute_err_miss(X, y, G, .35, runs = 10),
                    compute_err_miss(X, y, G, .5, runs = 10),
                    compute_err_miss(X, y, G, .65, runs = 10),
                    compute_err_miss(X, y, G, .8, runs = 10)])
miss_err.round(3)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

array([[0.002, 0.   ],
       [0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.005, 0.001]])

## Inosphere

In [ ]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
# print(data.head())
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
print(len(y))
X = np.delete(X,[0,1], axis = 1)
print(X.shape)
print(np.linalg.matrix_rank(X))

351
(351, 32)
32


In [ ]:
G = 2
nuclear_err = np.array([compute_err_nuclear(X, y, G, .2, runs = 10),
                    compute_err_nuclear(X, y, G, .35, runs = 10),
                    compute_err_nuclear(X, y, G, .5, runs = 10),
                    compute_err_nuclear(X, y, G, .65, runs = 10),
                    compute_err_nuclear(X, y, G, .8, runs = 10)])
nuclear_err.round(3)

In [ ]:
G = 2
miss_err = np.array([compute_err_miss(X, y, G, .2, runs = 10),
                    compute_err_miss(X, y, G, .35, runs = 10),
                    compute_err_miss(X, y, G, .5, runs = 10),
                    compute_err_miss(X, y, G, .65, runs = 10),
                    compute_err_miss(X, y, G, .8, runs = 10)])
miss_err.round(3)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

array([[0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.001],
       [0.007, 0.   ]])

In [ ]:
miss_err.round(3)

array([[0.003, 0.   ],
       [0.004, 0.   ],
       [0.005, 0.   ],
       [0.006, 0.001],
       [0.007, 0.   ]])

# seeds 

In [ ]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
                     sep = '\s+', header = None)
data = pd.DataFrame.to_numpy(data)
X,y = data[:,:7], data[:,7]-1 # reset the labels to go start from 0  
print(X.shape)
print(np.linalg.matrix_rank(X))

(210, 7)
7


In [ ]:
G = 3
nuclear_err = np.array([compute_err_nuclear(X, y, G, .2, runs = 10),
                    compute_err_nuclear(X, y, G, .35, runs = 10),
                    compute_err_nuclear(X, y, G, .5, runs = 10),
                    compute_err_nuclear(X, y, G, .65, runs = 10),
                    compute_err_nuclear(X, y, G, .8, runs = 10)])
nuclear_err.round(3)

In [ ]:
G = 3
miss_err = np.array([compute_err_miss(X, y, G, .2, runs = 10),
                    compute_err_miss(X, y, G, .35, runs = 10),
                    compute_err_miss(X, y, G, .5, runs = 10),
                    compute_err_miss(X, y, G, .65, runs = 10),
                    compute_err_miss(X, y, G, .8, runs = 10)])
miss_err.round(3)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

array([[0.005, 0.002],
       [0.007, 0.001],
       [0.01 , 0.002],
       [0.013, 0.002],
       [0.016, 0.004]])

# wine
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel()
print(X.shape)
print(np.linalg.matrix_rank(X))
# sum(y==0), sum(y==1), sum(y==2)

(178, 13)
13


In [ ]:
G = 3
nuclear_err = np.array([compute_err_nuclear(X, y, G, .2, runs = 10),
                    compute_err_nuclear(X, y, G, .35, runs = 10),
                    compute_err_nuclear(X, y, G, .5, runs = 10),
                    compute_err_nuclear(X, y, G, .65, runs = 10),
                    compute_err_nuclear(X, y, G, .8, runs = 10)])
nuclear_err.round(3)

In [ ]:
G = 3
miss_err = np.array([compute_err_miss(X, y, G, .2, runs = 10),
                    compute_err_miss(X, y, G, .35, runs = 10),
                    compute_err_miss(X, y, G, .5, runs = 10),
                    compute_err_miss(X, y, G, .65, runs = 10),
                    compute_err_miss(X, y, G, .8, runs = 10)])
miss_err.round(3)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2

array([[0.005, 0.   ],
       [0.008, 0.001],
       [0.011, 0.001],
       [0.013, 0.001],
       [0.018, 0.001]])

# Iris
The data set is also available in sklearn, as noted in the package's website. So, we load it directly from sklearn

In [ ]:
iris = datasets.load_iris()
X,y = iris.data, iris.target.ravel() 

In [ ]:
G = 3
nuclear_err = np.array([compute_err_nuclear(X, y, G, .2, runs = 10),
                    compute_err_nuclear(X, y, G, .35, runs = 10),
                    compute_err_nuclear(X, y, G, .5, runs = 10),
                    compute_err_nuclear(X, y, G, .65, runs = 10),
                    compute_err_nuclear(X, y, G, .8, runs = 10)])
nuclear_err.round(3)

In [ ]:
G = 3
miss_err = np.array([compute_err_miss(X, y, G, .2, runs = 10),
                    compute_err_miss(X, y, G, .35, runs = 10),
                    compute_err_miss(X, y, G, .5, runs = 10),
                    compute_err_miss(X, y, G, .65, runs = 10),
                    compute_err_miss(X, y, G, .8, runs = 10)])
miss_err.round(3)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

array([[0.009, 0.002],
       [0.014, 0.003],
       [0.024, 0.009],
       [0.033, 0.006],
       [0.043, 0.009]])